1. paths can be set from here.
2. all the paths are inside the dragvideo container
3. set kernel : Dragvideo env 

# PTI

## setting & functions

In [1]:
# ========================================
#            UTILITY FUNCTIONS
# ========================================

import os 

def add_dummy_config(filename,text,ROOT_PATH = '.',mode = "w"):
    file_path = os.path.join(ROOT_PATH, filename)
    with open(file_path, mode) as f:
        f.write(text)
        

def generate_text(**kwargs):
    text = "" 
    for key, value in kwargs.items():
        if type(value) == str:
            value = f"'{value}'"
        text += f"{key}= {value}\n"
    return text

def add_dummy_config_from_dict(filename,dict_,ROOT_PATH = '.',mode = "w"):
    text = generate_text(**dict_)
    add_dummy_config(filename,text,ROOT_PATH = ROOT_PATH,mode = mode)
    
    
# ========================================
def import_module_from_path( path,name='_module'):
    # ex: path = "path/to/module.py"
    import importlib.util
    spec = importlib.util.spec_from_file_location(name, path)
    module = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(module)
    return module


## set paths here

<b> NOTE:</b>  
To update the configs. this file changes must happen before import.
restart <i>to not import </i> from cached files

In [2]:
# imports 
import os
video_utils = import_module_from_path( "/home/bean/DragVideo/DragGAN/utils/video_utils.py")

# set path to PTI folder
PTI_PATH = '/home/bean/DragVideo/DragGAN/_PTI'
dummy_config_path = os.path.join(PTI_PATH, "configs","dummy")

input experiment path

In [3]:
# create experiment data folder structure
Experiment_name = 'exp_01_man'
Experiment_base_path = '/home/bean/DragVideo/Data_store/experiments/' 
Experiment_path = os.path.join(Experiment_base_path, Experiment_name)


import subprocess
os.chdir(PTI_PATH)   # .sh file is in PTI_PATH
subprocess.call(['sh', 'init_datadirs.sh', Experiment_name, Experiment_base_path]) 

Experiment directory already exists


1

In [4]:
# change path configs , hyperparameters 
paths_config_dict = {
    #pretrained models
    "e4e": "/home/bean/DragVideo/DragGAN/_PTI/pretrained_models/e4e_ffhq_encode.pt",
    "stylegan2_ada_ffhq": "/home/bean/DragVideo/DragGAN/_PTI/pretrained_models/ffhq.pkl",
    
    # to store tuned stylegan weights
    "checkpoints_dir": os.path.join(Experiment_path,'tuned_SG'),
    # to store latents
    "embedding_base_dir": os.path.join(Experiment_path,'latents'),
    # aligned / processed images
    "input_data_path": os.path.join(Experiment_path,'aligned'),
}

add_dummy_config_from_dict("paths_config.py", paths_config_dict,ROOT_PATH=dummy_config_path)

## preprocessing

In [5]:
# get frames 
video_path = "/home/bean/DragVideo/Data_store/OLD/original_videos/person_speaking_original.mp4"
raw_path = os.path.join(Experiment_path, "raw")

if False:
        video_utils.extract_frames(video_path, raw_path)

# align images
if False:
    os.chdir(PTI_PATH)
    from utils.align_data import pre_process_images
    pre_process_images(raw_path, IMAGE_SIZE=1024) # o/p: config.input_data_path

    

100%|██████████| 120/120 [02:11<00:00,  1.10s/it]


## running PTI

In [7]:
os.chdir(PTI_PATH)

use_multi_id_training = True

from scripts.run_pti import run_PTI
model_id = run_PTI(use_wandb=False, use_multi_id_training=use_multi_id_training)

Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]


/home/bean/DragVideo/env/Dragvideo/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/bean/DragVideo/env/Dragvideo/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Loading model from: /home/bean/DragVideo/env/Dragvideo/lib/python3.9/site-packages/lpips/weights/v0.1/alex.pth
current_directory:  /home/bean/DragVideo/DragGAN/_PTI
current_directory:  /home/bean/DragVideo/DragGAN/_PTI
torch.Size([3, 1024, 1024]) 3 1024 1024
Setting up PyTorch plugin "bias_act_plugin"... Done.


RuntimeError: The following operation failed in the TorchScript interpreter.
Traceback of TorchScript, serialized code (most recent call last):
  File "code/__torch__.py", line 107, in forward
      _48 = torch.__not__(torch.__contains__(_47, "conv1"))
      if _48:
        x5 = (_22).forward(x4, )
              ~~~~~~~~~~~~ <--- HERE
      else:
        x5 = x4
  File "code/__torch__.py", line 447, in forward
    _192 = torch.to(self.bias, ops.prim.dtype(x), False, False, None)
    _193 = self.padding
    x49 = torch.conv2d(x, _191, _192, [1, 1], [_193, _193], [1, 1], 1)
          ~~~~~~~~~~~~ <--- HERE
    x50 = __torch__.torch.nn.functional.relu(x49, False, )
    return x50

Traceback of TorchScript, original code (most recent call last):
  File "c:\p4research\research\tkarras\dnn\gan3support\feature_detectors\vgg.py", line 103, in forward
                    ys.append(x.to(torch.float32))
                if name not in ['pool5', 'fc1', 'fc2', 'fc3', 'softmax']:
                    